In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.7.0


In [3]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [4]:
# Loading all the required liberary
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch 

In [8]:
df = pd.read_csv('Real_Combine.csv')
print("This data set is all about predicting the air quality index \n",df)

This data set is all about predicting the air quality index 
          T    TM    Tm     SLP     H   VV     V    VM      PM 2.5
0      7.4   9.8   4.8  1017.6  93.0  0.5   4.3   9.4  219.720833
1      7.8  12.7   4.4  1018.5  87.0  0.6   4.4  11.1  182.187500
2      6.7  13.4   2.4  1019.4  82.0  0.6   4.8  11.1  154.037500
3      8.6  15.5   3.3  1018.7  72.0  0.8   8.1  20.6  223.208333
4     12.4  20.9   4.4  1017.3  61.0  1.3   8.7  22.2  200.645833
...    ...   ...   ...     ...   ...  ...   ...   ...         ...
1088  18.1  24.0  11.2  1015.4  56.0  1.8  15.9  25.9  288.416667
1089  17.8  25.0  10.7  1015.8  54.0  2.3   9.4  22.2  256.833333
1090  13.9  24.5  11.4  1015.0  95.0  0.6   8.7  14.8  169.000000
1091  16.3  23.0   9.8  1016.9  78.0  1.1   7.4  16.5  186.041667
1092  16.3  23.4   9.0  1017.3  68.0  1.3   7.8  18.3  185.583333

[1093 rows x 9 columns]


In [9]:
# splitting the data set
x = df.iloc[:,:-1] 
print(x)
y = df.iloc[:,-1]
print(y)

         T    TM    Tm     SLP     H   VV     V    VM
0      7.4   9.8   4.8  1017.6  93.0  0.5   4.3   9.4
1      7.8  12.7   4.4  1018.5  87.0  0.6   4.4  11.1
2      6.7  13.4   2.4  1019.4  82.0  0.6   4.8  11.1
3      8.6  15.5   3.3  1018.7  72.0  0.8   8.1  20.6
4     12.4  20.9   4.4  1017.3  61.0  1.3   8.7  22.2
...    ...   ...   ...     ...   ...  ...   ...   ...
1088  18.1  24.0  11.2  1015.4  56.0  1.8  15.9  25.9
1089  17.8  25.0  10.7  1015.8  54.0  2.3   9.4  22.2
1090  13.9  24.5  11.4  1015.0  95.0  0.6   8.7  14.8
1091  16.3  23.0   9.8  1016.9  78.0  1.1   7.4  16.5
1092  16.3  23.4   9.0  1017.3  68.0  1.3   7.8  18.3

[1093 rows x 8 columns]
0       219.720833
1       182.187500
2       154.037500
3       223.208333
4       200.645833
           ...    
1088    288.416667
1089    256.833333
1090    169.000000
1091    186.041667
1092    185.583333
Name: PM 2.5, Length: 1093, dtype: float64


In [22]:
"""
Hyper parameters
1. How many hidden layers should be present ?
2. How many number of neurons should be present in the hidden layers ?
3. What should be the learning rate ?

"""
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(units = hp.Int('units_' + str(i), min_value = 32, max_value = 512, step = 32), activation= 'relu'))
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3,1e-4])), loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])
    return model

In [30]:
from tensorflow.python.framework.ops import executing_eagerly_outside_functions
tuner = RandomSearch(
    build_model,
    objective= 'val_mean_absolute_error',
    max_trials = 5,
    executions_per_trial = 4,
    directory = 'project1',
    project_name = 'Air Quality index'
)

INFO:tensorflow:Reloading Oracle from existing project project1/Air Quality index/oracle.json
INFO:tensorflow:Reloading Tuner from project1/Air Quality index/tuner0.json


In [31]:
tuner.search_space_summary() 

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [32]:
 from sklearn.model_selection import train_test_split
 x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 40)
 

In [33]:
tuner.search(x_train, y_train, 
             epochs = 5, 
             validation_data = (x_test,y_test)
             )

INFO:tensorflow:Oracle triggered exit


In [29]:
tuner.results_summary()

Results summary
Results in project1/Air Quality index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 17
units_0: 256
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 11
units_0: 160
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 11
units_0: 288
learning_rate: 0.0001
Score: nan
Trial summary
Hyperparameters:
num_layers: 18
units_0: 224
learning_rate: 0.001
Score: nan
Trial summary
Hyperparameters:
num_layers: 5
units_0: 448
learning_rate: 0.0001
Score: nan
